# Example: Using OpenTelemetry SDK with Opik OTel API

This notebook demonstrates how to use any OpenTelemetry SDK to send traces to Opik. [OpenTelemetry](https://opentelemetry.io/) is a CNCF project that provides a standard way to collect distributed traces and metrics from applications.

Opik operates as an [OpenTelemetry Backend](https://www.comet.com/docs/opik/tracing/opentelemetry/overview) and maps the received traces to the Opik data model according to the OpenTelemetry Gen AI Conventions.

In this example, we'll use the [Python OpenTelemetry SDK](https://opentelemetry.io/docs/languages/python/) to send traces with GenAI attributes to Opik.


## Setup

In [ ]:
%pip install 'smolagents[telemetry]'

# Prepare Opik configuration and credentials

In [ ]:
import os
import getpass

OPIK_API_KEY = None
OPIK_PROJECT_NAME = "otel-python-sdk-integration"
OPIK_WORKSPACE = "lothiraldan"

if OPIK_API_KEY is None and "OPIK_API_KEY" not in os.environ:
    OPIK_API_KEY = getpass.getpass("Enter your OPIK API key: ")
elif OPIK_API_KEY is None:
    OPIK_API_KEY = os.environ["OPIK_API_KEY"]

OPIK_HOST = "https://www.comet.com/opik/"  # Opik Cloud
# OPIK_HOST = "https://<YOUR-OPIK-INSTANCE>/" # Opik self-hosted

headers = {}

if OPIK_API_KEY is not None:
    headers["Authorization"] = OPIK_API_KEY

if OPIK_PROJECT_NAME is not None:
    headers["projectName"] = OPIK_PROJECT_NAME

if OPIK_WORKSPACE is not None:
    headers["Comet-Workspace"] = OPIK_WORKSPACE

# Configure OpenTelemetry Python SDK

In [ ]:
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

provider = TracerProvider()
processor = BatchSpanProcessor(
    OTLPSpanExporter(
        endpoint=f"{OPIK_HOST}api/v1/private/otel/v1/traces",
        headers=headers,
    )
)
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
tracer = trace.get_tracer(__name__)

## Flattened attributes

Opentelemetry lets you attach a set of attributes to all spans by setting [`set_attribute`](https://opentelemetry.io/docs/languages/python/instrumentation/#add-attributes-to-a-span).

**GenAI Semantic Convention Attributes:**

In [ ]:
with tracer.start_as_current_span("GenAI Attributes") as span:
    span.set_attribute("gen_ai.prompt.0.role", "system")
    span.set_attribute(
        "gen_ai.prompt.0.content",
        "You are a coding assistant that helps write Python code.",
    )
    span.set_attribute("gen_ai.prompt.1.role", "user")
    span.set_attribute(
        "gen_ai.prompt.1.content",
        "Write a function that calculates the factorial of a number.",
    )

    span.set_attribute("gen_ai.completion.0.role", "assistant")
    span.set_attribute(
        "gen_ai.completion.0.content",
        """def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)""",
    )

    span.set_attribute("gen_ai.request.model", "gpt-4")
    span.set_attribute("gen_ai.request.temperature", 0.7)
    span.set_attribute("gen_ai.usage.prompt_tokens", 25)
    span.set_attribute("gen_ai.usage.completion_tokens", 45)